In [1]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()  

geemap package not installed. Installing ...
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=qS2mDgdQNl4A0vlbNLZA6clHC4Fmq1smK_QYIJeT5I8&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjLI3M9ohrmGP4qjA7XpxHETDoekmwGJWkuwPygF7Yos1mIBPnsE8g

Successfully saved authorization token.


In [ ]:

# Algorithm:
# 1. Get huc value for watershed of interest (see table below)
# 3. Create a new feature with selected watershed
# 5. Union the counties (if necessary), export the counties
# 6. Get the CDL data
# 7. Clip to the area of the watershed
# 8. Export 

#--------------------------------#
#   Name           |    huc8     #
#--------------------------------#
#   Lower Boise    |  17050114   #
#--------------------------------#




In [66]:
wshd = ee.FeatureCollection("USGS/WBD/2017/HUC08")

lbrb_wshd = wshd.filterMetadata('name', 'equals', 'Lower Boise')

#print to map
map = emap.Map()
map.addLayer(ee.Image().paint(lbrb_wshd, 0, 2), {}, 'Lower Boise River Basin')
map.addLayerControl()
map

In [67]:
dataset = ee.ImageCollection('USDA/NASS/CDL').filterBounds(lbrb_wshd)
lbrb_cropland = dataset.select('cropland') \
                    .map(lambda image: image.clip(lbrb_wshd))
croplandVis = {
  'min': 1.0,
  'max': 254.0,
  'palette': ['ffd300', #corn, dbl crop winwheat/corn, oats/corn, barley/corn
              #corn/soybeans
              'ff2626', #cotton, lettuce/cotton
              '00a832', #rice
              'ff9e0a', #sorghum, durum wheat/sorghum, barley/sorghum
              '267000', #soybeans, soybeans/cotton, soybeans/oats, barley/soybeans
              'ffff00', #sunflower
              '70a500', #peanuts
              '00af49', #tobacco, mustard, switchgrass
              'dda50a', #sweet corn
              '7cd3ff', #mint, herbs
              'e2007c', #Barley, lettuce/barley
              '896054', #durum wheat
              'd8b56b', #spring wheat
              'a57000', #winter wheat, winwheat/sorghum, winwheat/cotton
              'd69ebc', #other small grains, speltz, buckwheat, triticale
              '707000', #Dbl crop winwht/soybeans
              'aa007c', #rye
              'a05989',  #oats
              '700049', #millet
              'd1ff00', #canola, rape seed
              '7c99ff', #flaxseed
              'd6d600', #safflower
              'ffa5e2', #alfalfa
              'a5f28c', #other hay/non alfalfa
              '00af49', #camelina, other crops, chickpeas, hops, vetch
              'a800e2', #sugarbeets
              'a50000', #dry beans
              '702600', #potatoes, sweet potatoes
              'af7cff', #sugarcane
              'ff6666', #misc vegs and fruits, cucumbers, caneberries, watermelons
              #carrots, asparagus, garlic, cantelope, honeydew melons, broccoli,
              #peppers, greens, strawberries, squash, lettuce, pumpkins, cabbage
              #cauliflower, celery, radish, turnips, eggplant, gourds, cranberries
              'ffcc66', #onions
              '00ddaf', #lentils
              '54ff00', #peas
              'f2a377', #tomatoes
              'e8bfff', #clover/wildflowers
              'afffdd', #sod, grass seed
              'bfbf77', #fallow/ idle cropland
              '93cc93', #forest (deciduous, evergreen, mixed)
              'c6d69e', #shrubland
              'ccbfa3', #barren
              'ff00ff', #cherries
              'ff8eaa', #peaches, prunes, plums, nectarines, apricots
              'ba004f', #apples
              '704489', #grapes
              '007777', #Christmas trees
              'af9970', #other tree crops, pears, pomegranates
              'ffff7c', #citrus
              'b5705b', #pecans
              '00a582', #almonds
              'e8d6af', #walnuts
              'f2f2f2', #clouds/no data
              '999999', #developed, developed/open space, developed/low intensity
              #developed/med intensity, developed/high intensity
              '4970a3', #water, open water
              '7cafaf', #wetlands (woody and herbaceous)
              'e8ffbf', #nonag/undefined, grassland/pasture
              '00ffff', #aquaculture
              'd3e2f9', #perenial ice/snow
              '00ff8c', #pistachios
              '334933', #olives
              'e27026', #oranges
              '896054', #dbl crop lettuce/durum wheat
              '000099', #blueberries
  ]}

Map = emap.Map(center=(43.6150, -116.2023),zoom=8)
Map.addLayer(lbrb_cropland, croplandVis, 'Cropland')
Map


In [ ]:
downConfig = {'scale': 30, "maxPixels": 1.0E13, 'driveFolder': 'srp-nlcd'}  # scale means resolution.
img_lst = srb_landcover.toList(100)

count = srb_landcover.size().getInfo()

for i in range(0, count):
    image = ee.Image(img_lst.get(i))
    name = image.get('system:index').getInfo()
    print(name)
    task = ee.batch.Export.image(image.clip(srb_bnd.geometry()), name, downConfig)
    task.start()
    print(task.status())

NLCD1992
{'state': 'READY', 'description': 'NLCD1992', 'creation_timestamp_ms': 1602450996161, 'update_timestamp_ms': 1602450996161, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'D35E3TPVLPHPIXST7VZDLMRD', 'name': 'projects/earthengine-legacy/operations/D35E3TPVLPHPIXST7VZDLMRD'}
NLCD2001
{'state': 'READY', 'description': 'NLCD2001', 'creation_timestamp_ms': 1602451018960, 'update_timestamp_ms': 1602451018960, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'UNT7DBP4SQRBURSUVD466POV', 'name': 'projects/earthengine-legacy/operations/UNT7DBP4SQRBURSUVD466POV'}
NLCD2004
{'state': 'READY', 'description': 'NLCD2004', 'creation_timestamp_ms': 1602451040720, 'update_timestamp_ms': 1602451040720, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'MXGCVL6NHIIBKYX62RCAY43D', 'name': 'projects/earthengine-legacy/operations/MXGCVL6NHIIBKYX62RCAY43D'}
NLCD2006
{'state': 'READY', 'description': 'NLCD2006', 'creation_timestamp_ms': 1602451059662, 'update_timestamp

In [ ]:
task.status()

{'creation_timestamp_ms': 1602451143507,
 'description': 'NLCD2016',
 'id': 'VHMYVOWJI5VC7UR4CGUYWXAS',
 'name': 'projects/earthengine-legacy/operations/VHMYVOWJI5VC7UR4CGUYWXAS',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1602451143507}